# Parallel Impact-T run

This example shows how to set up the Impact-T input files and run script, but to leave the running to the user. 



In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
from impact import Impact, template_dir
import os

In [3]:
ifile = os.path.join(template_dir,'lcls_injector/ImpactT.in')
os.path.exists(ifile)

True

In [4]:
# Make Impact object
I = Impact(ifile, verbose=False, use_tempdir=False, workdir='$HOME/Scratch/temp')

# Change some things
I.input['header']['Np'] = 10000
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13
I.ele['stop_1']['s'] = 0.1

# Set parallel domain
I.input['header']['Npcol'] = 2
I.input['header']['Nprow'] = 2

# Turn on MPI switches
I.use_mpi=True
I.impact_bin = '$IMPACTT_MPI_BIN'
I.mpi_exe = 'mpirun' 

print(I)

================ Impact-T Summary ================    
10000 particles
1 bunch of electrons
total charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Processor domain: 2 x 2 = 4 CPUs
Space charge grid: 16 x 16 x 16
Maximum time steps: 1000000
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning
Impact-T configured in /Users/chrisonian/Scratch/temp


In [5]:
# Write input to workdir
I.write_input()
I.get_run_script()
# This will be the working path
I.path

'/Users/chrisonian/Scratch/temp'

# Run externally 
Parallel jobs often need to be run on special nodes and/or submitted to a queue, invoking the run script:

In [6]:
!cat /Users/chrisonian/Scratch/temp/run

mpirun -n 4 /Users/chrisonian/Code/impact/bin/ImpactTexe_mpi

Here we will do a quick run 

In [7]:
!cd $HOME/Scratch/temp;./run >log.txt

# Collect output

In [8]:
I.verbose=True
# Load output
I.load_output()

Loaded fort 30 : Fourth root of the fourth moments of the beam distribution
Loaded fort 25 : RMS Y information
Loaded fort 24 : RMS X information
Loaded fort 26 : RMS Z information
Loaded fort 28 : Load balance and loss diagnostics
Loaded fort 29 : Cube root of third moments of the beam distribution
Loaded fort 18 : Time and energy
Loaded fort 27 : Max amplitude information
Loaded fort 70 : Slice information of the final distribution
Loaded fort 60 : Slice information of the initial distribution
Loading particles
Loaded fort 40 : initial particle distribution at t = 0
Loaded fort 50 : final particle distribution projected to the centroid location of the bunch
Converting z to t according to cathode_kinetic_energy_ref = 1.0 eV
Converted initial_particles to ParticleGroup
Converted final_particles to ParticleGroup


# Alternative: load into new object
If this notebook went out of scope, we'd need to load the output into a new object

In [9]:
I2 = Impact('$HOME/Scratch/temp/ImpactT.in', use_tempdir=False)

================ Impact-T Summary ================    
10000 particles
1 bunch of electrons
total charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Processor domain: 2 x 2 = 4 CPUs
Space charge grid: 16 x 16 x 16
Maximum time steps: 1000000
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning

Configured to run in: /Users/chrisonian/Scratch/temp


In [10]:
I2.load_output()

Loaded fort 30 : Fourth root of the fourth moments of the beam distribution
Loaded fort 25 : RMS Y information
Loaded fort 24 : RMS X information
Loaded fort 26 : RMS Z information
Loaded fort 28 : Load balance and loss diagnostics
Loaded fort 29 : Cube root of third moments of the beam distribution
Loaded fort 18 : Time and energy
Loaded fort 27 : Max amplitude information
Loaded fort 70 : Slice information of the final distribution
Loaded fort 60 : Slice information of the initial distribution
Loading particles
Loaded fort 40 : initial particle distribution at t = 0
Loaded fort 50 : final particle distribution projected to the centroid location of the bunch
Converting z to t according to cathode_kinetic_energy_ref = 1.0 eV
Converted initial_particles to ParticleGroup
Converted final_particles to ParticleGroup


# Archiving
Archiving is the same as in the simple example

In [11]:
#import h5py

#H5FILE = os.path.expandvars('$HOME/Scratch/temp/output.h5')
#with h5py.File(H5FILE, 'w') as h5:
afile = os.path.expandvars('$HOME/Scratch/temp/output.h5')
I2.archive(afile)


Archiving to file /Users/chrisonian/Scratch/temp/output.h5


'/Users/chrisonian/Scratch/temp/output.h5'